In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.stats import chi2
import statsmodels.api as sm
from scipy.special import expit
np.set_printoptions(suppress=True)

In [15]:
address = 'C:/Users/moham/My Drive/Projects/Structural Econometrics Projects/Topics in Advanced Econometrics/4- GEV'
camping = pd.read_csv(address + '/camping.csv')
camping.rename(columns={'camper_id': 'id'}, inplace=True)

# Problem 1: Generalized Extreme Value Models

The data here is for camping choices. It is an individual level data. Each individual has 5 alternatives of parks to camp in. We have the choice of each individual in column visit. The data is in the long format. For each park, we have two attributes of time and cost. Also, we have two categories of parks. One, beach parks and the other is mountain parks. 

In [16]:
camping.head(5)

,id,park_id,park,visit,mountain,beach,cost,time
0,1,1,Mount Greylock,0,1,0,53.125051,204.096987
1,1,2,October Mountain,1,1,0,72.147801,262.922037
2,1,3,Horseneck Beach,0,0,1,97.150615,212.117346
3,1,4,Salisbury Beach,0,0,1,108.296025,181.254675
4,1,5,Scusset Beach,0,0,1,107.127049,151.808650


## a. Multinomial Logit model

Suppose the choice model is as follows:

$$
V_{nj} = \beta_1 C_{nj} + \beta_2 T_{nj} + \beta_3 M_j
$$

$M_j$ is a binary indicator of being in the mountain or not.
Here, we will use MLE to estimate the parameters. I will use the code from previous projects.

In [21]:
def loglikelihood(params, data):
    # Create a copy of the input DataFrame to avoid modifying the original one
    df = data.copy()
    # Independent variables as a NumPy array
    X = df[['cost', 'time', 'mountain']]
    # Calculating the exponential utility
    df['exp_Util'] = np.exp(np.dot(X, params))
    # Summing exp_Util for each id
    df['sum_exp_Util'] = df.groupby('id')['exp_Util'].transform('sum')
    # Probability calculation
    df['prob'] = df['exp_Util'] / df['sum_exp_Util']
    # Calculating the log-likelihood
    loglikelihood_value = np.sum(df['visit'] * np.log(df['prob']))
    # Return negative log-likelihood (for minimization)
    return -loglikelihood_value

In [ ]:
initial_params = np.zeros(3)
result = minimize(loglikelihood, 
                  initial_params, 
                  args=(camping[['id', 'visit', 'time', 'cost', 'mountain']],), 
                  method='BFGS', 
                  options={'disp': False})
parameters = result.x

In [26]:
parameters

array([-0.01478385, -0.00163202, -0.33104973])

In [28]:
std = np.sqrt(result.hess_inv.diagonal())
z_stat = parameters / std
p_values = 2 * (1 - norm.cdf(abs(z_stat)))
summary_table = pd.DataFrame({
    'Parameter': ['Cost', 'time', 'Mountain'],
    'Coefficient': parameters,
    'Standard Error': std,
    'Z-statistic': z_stat,
    'P-value': p_values
})
print(summary_table.round(3).to_string(index=False))

Parameter  Coefficient  Standard Error  Z-statistic  P-value
     Cost       -0.015           0.004       -3.914    0.000
     time       -0.002           0.000       -3.929    0.000
 Mountain       -0.331           0.180       -1.839    0.066


The campers in our dataset obtain less utility, ceteris paribus, from camping in the mountains than from camping at the beach. Also, as expected, the marginal utilities of both cost and time are negative.

### Dollor value of time and mountains

In [31]:
dollor_value = parameters[0]
value_time = parameters[1]/parameters[0] * 60
value_mountain = parameters[2]/parameters[0]
print('Value of Time:', value_time.round(3), '$')
print('Value of Mountain:', value_mountain.round(3), '$')

Value of Time: 6.624 $
Value of Mountain: 22.393 $


So, campers value going to the mountain 22 dollor less than going to a beach.